# First Models

In [1]:
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer

# LSTM layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [13]:
train_df = pd.read_csv('data/train.csv')
# Selecting Edgar Allen Poe as author style to emulate
author = train_df[train_df['author'] == 'EAP']["text"]
print('Number of sentences: ',author.shape[0])


Number of sentences:  7900


# Tokenize words in corpus using Keras Tokenizer.
This assigns a unique integer to each word and then replaces all instances of that word with the integer. Tokenization is necessary for preparing data for embedding layer ( see below )

In [40]:
max_words = 50000 # Max size of the dictionary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(author.values)
sequences = tok.texts_to_sequences(author.values)
print(sequences[:5])

[[19, 2397, 80, 1001, 29, 31, 177, 2, 4073, 1, 1960, 2, 11, 3024, 15, 7, 110, 157, 41, 2146, 3, 481, 4, 1, 149, 2147, 7, 393, 74, 114, 101, 439, 2, 1, 162, 32, 913, 6453, 136, 1, 380], [6, 21, 142, 150, 10, 5, 551, 2148, 319, 28, 16, 15, 20, 8999, 128, 1, 3025, 2398, 30, 171, 2, 1797, 697, 20, 180, 2148, 6454, 12, 33, 188, 2, 1, 869, 243, 522, 1264], [1, 6455, 203, 14, 19, 149, 180, 6456, 6, 1, 1357, 2, 1358, 9000, 3, 83, 2149, 10, 355, 140, 794], [1, 4074, 491, 6, 9001, 28, 11, 158], [7, 287, 9, 36, 48, 22, 73, 4, 644, 9002, 114, 101, 346, 4, 271, 2, 9003, 3, 81, 2, 1, 3026, 2, 6457, 3, 282, 53, 34, 6458, 19, 339, 22, 43, 97, 608, 7, 450, 4, 36, 133, 1191, 88, 12, 133, 71, 914, 1, 759, 3027, 2, 9, 1445, 1359, 18, 760, 12, 4973, 6, 1, 421, 9004, 9005, 7, 214, 9, 36, 48, 22, 3449, 3028, 98, 124, 1192, 4, 1, 92, 9006, 6, 3450, 3, 7, 761, 870, 9, 36, 55, 111, 32]]


In [45]:
# Flatten the list of lists resulting from the tokenization. This will reduce the list
# to one dimension, allowing us to apply the sliding window technique to predict the next word
text = [item for sublist in sequences for item in sublist]
vocab_size = len(tokenizer.word_index)

In [46]:
print('Vocabulary size in this corpus: ', vocab_size)

Vocabulary size in this corpus:  15713


In [43]:
# Training on 19 words to predict the 20th
sentence_len = 20
pred_len = 1
train_len = sentence_len - pred_len
seq = []
# Sliding window to generate test and train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])
# Reverse dictionary so as to decode tokenized sequences back to words and sentences
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# dump(tok, open('tokenizer.pkl', 'wb'))

In [44]:
trainX = []
trainy = []
for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

# Model Architecture:
1. Embedding layer
    - Helps model understand 'meaning' of words by mapping them to representative vector space instead of semantic integers
2. Stacked LSTM layers
    - Stacked LSTMs add more depth than additional cells in a single LSTM layer (see: https://machinelearningmastery.com/stacked-long-short-term-memory-networks/)
3. Dense (regression) layer
4. Dense layer with Softmax activation 
    - Outputs word probability across entire vocab

In [51]:
# define model
model = Sequential([
    Embedding(vocab_size+1, 50, input_length=train_len),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 19, 50)            785700    
_________________________________________________________________
lstm_4 (LSTM)                (None, 19, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 15713)             1587013   
Total params: 2,523,613
Trainable params: 2,523,613
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(np.asarray(trainX),
         pd.get_dummies(np.asarray(trainy)),
         epochs = 500,
         batch_size = 10240,
         callbacks = callbacks_list,
         verbose = 2)

In [56]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(np.asarray(trainX), pd.get_dummies(np.asarray(trainy)), batch_size=128, epochs=100)


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
201082/201082 [==============================] - 242s 1ms/sample - loss: 6.8433 - acc: 0.0803
Epoch 2/100
201082/201082 [==============================] - 246s 1ms/sample - loss: 6.3949 - acc: 0.1045
Epoch 3/100
201082/201082 [==============================] - 241s 1ms/sample - loss: 6.1931 - acc: 0.1163
Epoch 4/100
201082/201082 [==============================] - 242s 1ms/sample - loss: 6.0466 - acc: 0.1245
Epoch 5/100
201082/201082 [==============================] - 243s 1ms/sample - loss: 5.9306 - acc: 0.1302
Epoch 6/100
201082/201082 [==============================] - 243s 1ms/sample - loss: 5.8292 - acc: 0.1351
Epoch 7/100
201082/201082 [==============================] - 245s 1ms/sample - loss: 5.7382 - acc: 0.1388
Epoch 8/100
201082/201082 [==============================] - 255s 1ms/sample - loss: 5.6575 - acc: 0.1414
Epoch 9/100
201082/201082 [==============================] - 237s 1ms/sample - loss: 5.5819 - acc: 0.144

201082/201082 [==============================] - 231s 1ms/sample - loss: 3.4813 - acc: 0.3344
Epoch 76/100
201082/201082 [==============================] - 231s 1ms/sample - loss: 3.4604 - acc: 0.3378
Epoch 77/100
201082/201082 [==============================] - 231s 1ms/sample - loss: 3.4504 - acc: 0.3385
Epoch 78/100
201082/201082 [==============================] - 259s 1ms/sample - loss: 3.4325 - acc: 0.3404
Epoch 79/100
201082/201082 [==============================] - 242s 1ms/sample - loss: 3.4173 - acc: 0.3433
Epoch 80/100
201082/201082 [==============================] - 246s 1ms/sample - loss: 3.3962 - acc: 0.3461
Epoch 81/100
201082/201082 [==============================] - 257s 1ms/sample - loss: 3.3840 - acc: 0.3483
Epoch 82/100
201082/201082 [==============================] - 242s 1ms/sample - loss: 3.3629 - acc: 0.3515
Epoch 83/100
201082/201082 [==============================] - 255s 1ms/sample - loss: 3.3489 - acc: 0.3533
Epoch 84/100
201082/201082 [======================

In [58]:
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
model.save('model_weights.hdf5')

In [59]:
# model.compile(optimizer=Adam(lr=0.001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# filepath = "./weight_tr5.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]
# history = model.fit(np.asarray(trainX),
#          pd.get_dummies(np.asarray(trainy)),
#          epochs = 500,
#          batch_size = 10240,
#          callbacks = callbacks_list,
#          verbose = 2)

In [64]:
def gen(seq,max_len = 20):
    sent = tokenizer.texts_to_sequences([seq])
    #print(sent)
    while len(sent[0]) < max_len:
        sent2 = pad_sequences(sent[-19:],maxlen=19)
        op = model.predict(np.asarray(sent2).reshape(1,-1))
        sent[0].append(op.argmax()+1)
    return " ".join(map(lambda x : reverse_word_map[x],sent[0]))

In [65]:
test1='There was a big bear in the corner.'

In [67]:
gen(test1,100)

'there was a big bear in the corner of the earth and the most practice is in the same moment when the whole town was topsy massa de la unus la bedloe deposes that in the whole of the most practice that had been in the world of being sure of the most indefinite sense of air is excessively early and sure but the most rarefied and ferry of be apprehend you to bishop in determining by the dust in a very degree the moon and the most practice of all who is not altogether the whole of the most'